<a href="https://colab.research.google.com/github/spencerklawans/466-Project/blob/main/WildfirePrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wildfire Prediction
Brought to you by Gooogle Earth Engine

In [1]:
# Import libraries.
import ee
import folium

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = "Map Data © Google Earth Engine",
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=TpihwL0i342Y-BmIm6Ejwd9G-NT6YnuSsY61-d5uS_s&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g7QISmivcLS55sXribT8R1JmYf4oCmaSLcDVm1hegpAUkPXtxG-Wcc

Successfully saved authorization token.


In [ ]:
#Init KDD libraries

%matplotlib inline

import numpy as np
import pandas as pd
import sklearn

In [46]:
California = ee.Geometry.Rectangle(-124.18, 41.39, -114.35, 32.75)

# Import our fire events as features
modis_fires = ee.FeatureCollection('JRC/GWIS/GlobFire/v2/FinalPerimeters').filterBounds(California)
# filter for only fires with greater than 10,000,000 square meters
modis_fires = modis_fires.filter(ee.Filter.gt('area', 10000000))

In [47]:
slo = ee.Geometry.Point([-120.4358, 35.3102])

# Import ImageCollections
landfire_prs = ee.Image("LANDFIRE/Fire/PRS/v1_2_0/CONUS")
landfire_evc = ee.Image("LANDFIRE/Vegetation/EVC/v1_4_0/CONUS")

In [48]:
# Set visualization parameters. I'm pretty sure min and max should track to the min and max values of the set at https://developers.google.com/earth-engine/datasets/catalog/LANDFIRE_Fire_PRS_v1_2_0#bands
prs_vis_params = {
    'min': 1,
    'max': 133,
}
evc_vis_params = {
    'min': 11,
    'max': 172,
}

# Create a folium map object.
my_map = folium.Map(location=[35.3102, -120.4358], zoom_start=8)

# Add the fire severity & existing vegetation type images to the map object.
my_map.add_ee_layer(landfire_prs, prs_vis_params, 'Percent Replacement-Severity Fire')
my_map.add_ee_layer(landfire_evc, evc_vis_params, 'Existing Vegetation Cover')
my_map.add_ee_layer(ee.Image().paint(modis_fires), {'palette': '#FF0000'}, 'Fire Events')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)